In [231]:
import matplotlib.pyplot as plt
import pandas as pd
import re

In [232]:
vote_counts_state = pd.read_csv("datasets/vote_counts_bystate.csv")
vote_counts_district = pd.read_csv("datasets/vote_counts_bydistrict.csv")
demo_data = pd.read_csv("datasets/ACSDP1Y2018.DP05_data_with_overlays_2020-09-15T163409.csv", header=1, encoding="latin1")

In [233]:
# The cell where we define helper functions
def check_regex(search_string, full_string):
    if re.search(search_string, full_string) is not None:
        return True
    return False

def search_columns(search_string, dataframe):
    matches = []
    for column in dataframe.columns:
        if(check_regex(search_string, column)):
           matches.append(column)
    return matches

def extract_district_number(string):
    # Regex match to get district number
   return int(re.search("([0-9]+){1}", string).group())

def extract_state_name(string):
    # Regex match + slice to get state name
    return re.search(", ([A-Za-z])*", string).group()[2:]

In [234]:
# The cell where we apply the helpers and drop unwanted columns

# Add state and district for joining with vote result columns
demo_data["state"] = demo_data["Geographic Area Name"].apply(extract_state_name)
demo_data["district"] = demo_data["Geographic Area Name"].apply(extract_district_number)

# Drop margins and percent margins
margins = search_columns("Margin of Error!!", demo_data)
demo_data = demo_data.drop(margins, axis=1)
percent_margins = search_columns("Percent Margin of Error!!", demo_data)
demo_data = demo_data.drop(percent_margins, axis=1)

In [235]:
# Select only race columns
race_cols = search_columns("Estimate!!RACE", demo_data)
demo_data_race = demo_data[race_cols + ["state", "district"]]
demo_data_race.columns

Index(['Estimate!!RACE!!Total population',
       'Percent Estimate!!RACE!!Total population',
       'Estimate!!RACE!!Total population!!One race',
       'Percent Estimate!!RACE!!Total population!!One race',
       'Estimate!!RACE!!Total population!!Two or more races',
       'Percent Estimate!!RACE!!Total population!!Two or more races',
       'Estimate!!RACE!!Total population!!One race.1',
       'Percent Estimate!!RACE!!Total population!!One race.1',
       'Estimate!!RACE!!Total population!!One race!!White',
       'Percent Estimate!!RACE!!Total population!!One race!!White',
       'Estimate!!RACE!!Total population!!One race!!Black or African American',
       'Percent Estimate!!RACE!!Total population!!One race!!Black or African American',
       'Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native',
       'Percent Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native',
       'Estimate!!RACE!!Total population!!One race!!American I

In [241]:
# Join race data and voting results
vote_counts_district_2d = vote_counts_district.set_index(["state", "district"])
vote_counts_district_2d
demo_data_2d = demo_data_race.set_index(["state", "district"])
demo_data_2d

# Generate the working dataframe for data analysis
vote_counts_race = vote_counts_district_2d.join(demo_data_2d, how="inner")

In [242]:
vote_counts_race

dem_votes  rep_votes winner  wasted_democrat  \
state     district                                                 
Alabama   1           89226.0   153228.0      r          89226.0   
          2           86931.0   138879.0      r          86931.0   
          3           83996.0   147770.0      r          83996.0   
          4           46492.0   184255.0      r          46492.0   
          5          101388.0   159063.0      r         101388.0   
...                       ...        ...    ...              ...   
Wisconsin 4          206487.0    59091.0      d          73697.0   
          5          138385.0   225619.0      r         138385.0   
          6          144536.0   180311.0      r         144536.0   
          7          124307.0   194061.0      r         124307.0   
          8          119265.0   209410.0      r         119265.0   

                    wasted_republican  Estimate!!RACE!!Total population  \
state     district                                                        
Alabama   1                   32000.0                            715346   
          2                   25973.0                            678122   
          3                   31886.0                            708409   
          4                   68880.5                            686297   
          5                   28836.5                            725634   
...                               ...                               ...   
Wisconsin 4                   59091.0                            710573   
          5                   43616.0                            731341   
          6                   17886.5                            714886   
          7                   34876.0                            710420   
          8                   45071.5                            735997   

                    Percent Estimate!!RACE!!Total population  \
state     district                                             
Alabama   1                                           715346   
          2                                           678122   
          3                                           708409   
          4                                           686297   
          5                                           725634   
...                                                      ...   
Wisconsin 4                                           710573   
          5                                           731341   
          6                                           714886   
          7                                           710420   
          8                                           735997   

                    Estimate!!RACE!!Total population!!One race  \
state     district                                               
Alabama   1                                             702333   
          2                                             663115   
          3                                             695576   
          4                                             670732   
          5                                             706273   
...                                                        ...   
Wisconsin 4                                             679706   
          5                                             717464   
          6                                             701406   
          7                                             696490   
          8                                             719036   

                    Percent Estimate!!RACE!!Total population!!One race  \
state     district                                                       
Alabama   1                                                      98.2    
          2                                                      97.8    
          3                                                      98.2    
          4                                                      97.7    
          5            